In [1]:
import pandas as pd
import re

In [2]:
df = pd.read_excel(r'd:\Onedrive\OneDrive - UNI CONSULTING\Attachments\Code\uni_craw_tvpl\uni_craw_section_chapter_header\Craw_ko_en\ko_en_section_chapter_heading_des_notes.xlsx')

In [3]:
def create_section_df(df):
    # Select columns: 'ensection', 'ensection_note', 'kosection_note'
    df = df.loc[:, ['ensection', 'enheading_notes', 'koheading_notes']]
    # Drop duplicate rows across all columns
    df = df.drop_duplicates()
    return df

# Loaded variable 'df' from URI: d:\Onedrive\OneDrive - UNI CONSULTING\Attachments\Code\uni_craw_tvpl\uni_craw_section_chapter_header\Craw_ko_en\ko_en_section_chapter_heading_des_notes.xlsx

heading_df = create_section_df(df.copy())

In [40]:
from bs4 import BeautifulSoup

def fix_atag(html):
    # Phân tích cú pháp HTML
    soup = BeautifulSoup(html, 'html.parser')

    # Duyệt qua tất cả thẻ <a>
    for a_tag in soup.find_all('a'):
        # Kiểm tra nếu thẻ <a> chứa <img>
        if a_tag.find('img'):
            del a_tag['rel']
        else:
            # Thay thẻ <a> bằng text bên trong nó
            a_tag.replace_with(a_tag.text)

    # Kết quả sau khi xử lý
    result = str(soup).replace('"/clip/','"https://unipass.customs.go.kr/clip/')
    # print(result)
    return result

In [35]:
def process_section_notes(df,en_col,ko_col):
    
    def split_text(text):
        if not isinstance(text, str):
            return []
        text = text.replace('\n \n', '\n\n')
        text = text.replace('_x000D__x000D_', '\n')
        # Thay thế 3 hoặc nhiều '\n' thành '\n\n'
        text = re.sub(r'\n{2,}', '\n', text)
        delimiter = '\n'
        parts = text.split(delimiter)
        # for part in parts:
            # print(part) 
        return parts
    
    def get_heading_note_index(parts):
        pattern = r"^\d{4}\.\d{2}\s*-*"
        matches = [i for i, text in enumerate(parts) if re.match(pattern, text)]
        # Lấy index lớn nhất nếu có
        if matches:
            largest_index = max(matches)
            # print(f"Vị trí có index lớn nhất là: {largest_index}, chuỗi: {parts[largest_index]}")
            return largest_index
        else:
            # print("Không tìm thấy chuỗi phù hợp với pattern.")
            return 1

    def process_notes(x):
        split = split_text(x)
        note_idx = get_heading_note_index(split)
        if note_idx == 1:
            return split[1:]
        else :
            return split[note_idx+1:]    

    # Áp dụng hàm
    df['heading_name'] = df[en_col].apply(lambda x: split_text(x)[0])
    df['en_notes'] = df[en_col].apply(process_notes)
    df['ko_notes'] = df[ko_col].apply(process_notes)
    
    return df


In [41]:
# Gọi hàm xử lý
heading_df_2 = process_section_notes(heading_df, 'enheading_notes', 'koheading_notes')


In [42]:
heading_df_2['ko_notes'] = heading_df_2['ko_notes'].apply(lambda x: [fix_atag(html) for html in x])

C:\Users\nguye\AppData\Local\Temp\ipykernel_18952\2365387520.py:5: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  soup = BeautifulSoup(html, 'html.parser')


In [43]:
heading_df_2

,ensection,enheading_notes,koheading_notes,heading_name,en_notes,ko_notes
0,1,"01.01 Live horses, asses, mules and hinnies ...",01.01 - 살아 있는 말ㆍ당나귀ㆍ노새ㆍ버새(+)\n\n- 말\n\n0101.21...,"01.01 Live horses, asses, mules and hinnies ...","[This heading covers horses (mares, stallions,...",[이 호에는 말(암컷의 말․번식용 말․거세한 말․새끼말․조랑말)․당나귀․노새와 버새...
1,1,01.02 Live bovine animals (+).\n\n - Cattle ...,01.02 - 살아 있는 소(+)\n\n- 축우(畜牛)\n \n0102.21 - 번...,01.02 Live bovine animals (+).,"[(1) Cattle :, This category covers bovine ...","[(1) 축우(畜牛 : cattle), 이 범주에는 보스(Bos)속의 소과 동..."
2,1,01.03 - Live swine (+).\n\n0103.10 - Pure bred...,01.03 - 살아 있는 돼지(+)\n\n0103.10 - 번식용\n\n- 기타\n...,01.03 - Live swine (+).,[This heading covers both domestic pigs and wi...,"[이 호에는 가축용 돼지와 야생돼지(예: 멧돼지)를 분류한다., ◦, ◦ ◦, [..."
3,1,01.04 - Live sheep and goats. \n\n0104.10 - Sh...,01.04 - 살아 있는 면양과 염소\n\n0104.10 - 면양\n\n0104.2...,01.04 - Live sheep and goats.,[This heading covers domestic or wild sheep (r...,[이 호에는 가축이나 야생 면양(수컷․암컷과 어린양)과 가축용 염소나 야생의 염소와...
4,1,"01.05 - Live poultry, that is to say, fowls of...",01.05 - 살아 있는 가금(家禽)류[닭(갈루스 도메스티쿠스(Gallus dome...,"01.05 - Live poultry, that is to say, fowls of...",[This heading covers only live domestic birds ...,[이 호에는 오직 여기에 열거된 종류의 살아 있는 가금(家禽 : poultry)만을...
...,...,...,...,...,...,...
1222,21,"97.02 Original engravings, prints and lithog...",97.02 - 오리지널 동판화ㆍ목판화ㆍ석판화\n \n9702.10 - 제작 후 10...,"97.02 Original engravings, prints and lithog...","[This heading covers original engravings, prin...",[이 호에는 오리지널 동판화ㆍ목판화ㆍ석판화를 분류한다(고대의 것인지 현대의 것인지에...
1223,21,"97.03 Original sculptures and statuary, in a...",97.03 - 오리지널 조각과 조상(彫像)(어떤 재료라도 가능하다)\n\n9703....,"97.03 Original sculptures and statuary, in a...",[This heading covers original sculptures and s...,[이 호에는 오리지널 조각(sculpture)과 조상(彫像 : statuary)을 ...
1224,21,"97.04 Postage or revenue stamps, stamp-postm...",97.04 - 우표ㆍ수입인지ㆍ우편요금 별납증서ㆍ초일(初日)봉투ㆍ우편엽서류와 이와 유...,"97.04 Postage or revenue stamps, stamp-postm...",[This heading covers the following products us...,[이 호에는 제4907호의 것을 제외하고 사용된 것인지에 상관없이 다음의 것을 분류...
1225,21,97.05 Collections and collectors’ pieces of ...,97.05 - 수집품과 표본[고고학ㆍ민족학ㆍ사학ㆍ동물학ㆍ식물학ㆍ광물학ㆍ해부학ㆍ고생물...,97.05 Collections and collectors’ pieces of ...,[These articles are very often of little intri...,[이 호의 물품은 본질적인 가치는 적지만 그 희소성ㆍ집합(their grouping...
